# Deployment

## Set up and configure connection to SDK

In [36]:
import datarobot as dr #needs 2.25 or later
import pickle

In [37]:
dr.Client(config_path = '/Volumes/GoogleDrive/My Drive/Configurations/drconfig.yaml')

## Load model

In [38]:
deploymentModelPickle = open('deploymentModel.pickle','rb')
deploymentModel = pickle.load(deploymentModelPickle)
deploymentModelPickle.close()

In [39]:
project = deploymentModel['project']
model = deploymentModel['model']

## Create deployment

In [40]:
deploymentName = project.project_name

In [41]:
prediction_server = dr.PredictionServer.list()[0]

In [44]:
deployment = dr.Deployment.create_from_learning_model(
    model.id, label=deploymentName, description='Orsted',
    default_prediction_server_id=prediction_server.id)
deployment

Deployment(Fault prediction from sensor data Orsted AUTO)

## Show all deployments

In [45]:
deployments = dr.Deployment.list()
deployments

[Deployment(Demo Probability of Default (Risk Management)),
 Deployment(Risk Management - Custom Models),
 Deployment(MMM Twitter messages polarity monitoring in Boston),
 Deployment(Medical Shipment Delivery),
 Deployment(Server Response Time Prediction [Challengers][Custom Model Champion]),
 Deployment(MBTA Parking Utilization Forecasting [Deprecated]),
 Deployment([RAM-MODEL] Model Fit RAM Predictions),
 Deployment([MultiSeries] Bluebikes Availability (with challengers)),
 Deployment(Lending Club Deployed Model),
 Deployment(GCP MLOps Agent [Remote Model]),
 Deployment(Loan Defaults with Challengers [Segments]),
 Deployment(Housing Price Predictor (with Challengers)),
 Deployment(is_bad Predictions),
 Deployment(readmitted Predictions for App builder lab and Continuous AI Lab),
 Deployment(Wine Quality),
 Deployment(salary Predictions),
 Deployment(target Predictions),
 Deployment(Lending Club API with latest transactions),
 Deployment(Fault prediction from sensor data Orsted AUTO),

Let's filter for our own deployment

In [46]:
filters = dr.models.deployment.DeploymentListFilters(
    role='OWNER',
    #accuracy_health=dr.enums.DEPLOYMENT_ACCURACY_HEALTH_STATUS.FAILING
)
deployments = dr.Deployment.list(filters=filters)
deployments


[Deployment(is_bad Predictions),
 Deployment(readmitted Predictions for App builder lab and Continuous AI Lab),
 Deployment(Wine Quality),
 Deployment(salary Predictions),
 Deployment(target Predictions),
 Deployment(Lending Club API with latest transactions),
 Deployment(Fault prediction from sensor data Orsted AUTO),
 Deployment(Fault prediction from sensor data Orsted AUTO)]

### Extra configurations

Check whether drift and accuracy tracking is turned on

In [47]:
settings = deployment.get_drift_tracking_settings()
settings

{'target_drift': {'enabled': False}, 'feature_drift': {'enabled': False}}

In [48]:
deployment = dr.Deployment.get(deployment_id=deployment.id)
deployment.update_drift_tracking_settings(target_drift_enabled=True, feature_drift_enabled=True)

Now tracking is turned on

## Predict

This can be done through
- low-latency RESTful API
- batch prediction API

From within the DataRobot CLI, you can also perform batch predictions directly with `model.request_predictions`:

In [29]:
dataset = project.upload_dataset(sourcedata='Data/' +"test.csv")

In [51]:
pred_job = model.request_predictions(dataset.id)
preds = pred_job.get_result_when_complete()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/datarobot/utils/__init__.py:339: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  return frame.drop("prediction_values", 1)


In [52]:
preds

,row_id,prediction,positive_probability,prediction_threshold,class_0.0,class_1.0
0,0,1.0,0.970532,0.5,0.029468,0.970532
1,1,1.0,0.960706,0.5,0.039294,0.960706
2,2,0.0,0.017368,0.5,0.982632,0.017368
3,3,0.0,0.001075,0.5,0.998925,0.001075
4,4,0.0,0.000969,0.5,0.999031,0.000969
5,5,0.0,0.001822,0.5,0.998178,0.001822
6,6,1.0,0.997659,0.5,0.002341,0.997659
7,7,0.0,0.002516,0.5,0.997484,0.002516
8,8,0.0,0.011136,0.5,0.988864,0.011136


## Monitoring

### service stats

In [53]:
from datarobot.models import Deployment, ServiceStats

deployment_id = deployment.id

# call `get` functions on a `Deployment` object
deployment = Deployment.get(deployment_id)
service_stats = deployment.get_service_stats()

# directly fetch without a `Deployment` object
service_stats = ServiceStats.get(deployment_id)

In [54]:
service_stats.metrics

{'totalPredictions': 0,
 'totalRequests': 0,
 'slowRequests': 0,
 'executionTime': None,
 'responseTime': None,
 'userErrorRate': 0,
 'serverErrorRate': 0,
 'numConsumers': 0,
 'cacheHitRatio': 0,
 'medianLoad': 0,
 'peakLoad': 0}

### Data drift

In [55]:
from datetime import datetime
from datarobot.enums import DATA_DRIFT_METRIC
from datarobot.models import Deployment, FeatureDrift

In [56]:
deployment = Deployment.get(deployment_id=deployment.id)
target_drift = deployment.get_target_drift(
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15)
)
target_drift.drift_score

Use `DATA_DRIFT_METRIC.ALL` to retrieve a list of supported metrics

In [57]:
DATA_DRIFT_METRIC.ALL

['psi', 'kl_divergence', 'dissimilarity', 'hellinger', 'js_divergence']

In [58]:
feature_drift_data = FeatureDrift.list(
    deployment_id=deployment.id,
    start_time=datetime(2019, 8, 1, hour=15),
    end_time=datetime(2019, 8, 8, hour=15),
    metric=DATA_DRIFT_METRIC.PSI
)
feature_drift = feature_drift_data[0]
feature_drift.name

'biSecondlyReadings[sensor_q1] (5 minutes median)'

In [59]:
feature_drift.drift_score

### Accuracy

This is also possible, by providing true actuals, and matching them with `association_ID`

### Challenger / champion

Can be enabled and works without any effort from the user side

## Notes

At prediction time only the primary dataset is given as input.

Therefore, it is possible to replace the secondary dataset with commands including

In [68]:
default_dataset_config = dr.SecondaryDatasetConfigurations.list(project_id=project.id)[0]
config = default_dataset_config.to_dict().copy()
config

{'id': '6195218e5e93cfbce7635929',
 'project_id': '61951f09863a78bbdd6a4161',
 'config': [{'feature_engineering_graph_id': '61951fe55e93cfbce7635920',
   'secondary_datasets': [{'identifier': 'biSecondlyReadings',
     'catalog_id': '6194d3d8e77712a1426a40f8',
     'catalog_version_id': '6194d3d8e77712a1426a40f9',
     'snapshot_policy': 'latest'}]}],
 'name': 'Default Configuration',
 'secondary_datasets': [{'identifier': 'biSecondlyReadings',
   'catalog_id': '6194d3d8e77712a1426a40f8',
   'catalog_version_id': '6194d3d8e77712a1426a40f9',
   'snapshot_policy': 'latest'}],
 'creator_full_name': 'Toon Weyens',
 'creator_user_id': '6156cfd4c5145f3407af633b',
 'created': datetime.datetime(2021, 11, 17, 15, 36, 46, tzinfo=tzutc()),
 'featurelist_id': None,
 'credential_ids': [],
 'is_default': True,
 'project_version': None}

In [76]:
default_dataset_config.secondary_datasets[0].to_dict()

{'identifier': 'biSecondlyReadings',
 'catalog_id': '6194d3d8e77712a1426a40f8',
 'catalog_version_id': '6194d3d8e77712a1426a40f9',
 'snapshot_policy': 'latest'}

After this, take a new shapshot for the replacement dataset, and create a config for this. Then upload it using
`dataset = project.upload_dataset(sourcedata, secondary_datasets_config_id=new_secondary_dataset.id)`
